In [2]:
## I don't think that I use all of these libraries anymore
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup


In [13]:
# URL to scrape
url = "https://www.caringbridge.org/site/6f33ada9-525c-3ce3-be6f-34b647b78d2d" # the whole directory. Note that this view only exists
# because I "memorialized" the website
#url = "https://www.caringbridge.org/site/6f33ada9-525c-3ce3-be6f-34b647b78d2d/post/953dde50-3c45-35f3-9f19-20e51d9961a7" # One subpage

In [30]:
url

'https://www.caringbridge.org/site/6f33ada9-525c-3ce3-be6f-34b647b78d2d'

In [ ]:

# Set up Chrome options to use your profile
chrome_options = Options()
chrome_options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"  # Correct path
chrome_options.add_argument("/Users/joelswenson/Library/Application Support/Google/Chrome")  # Update this path
chrome_options.add_argument("profile-directory=Default")  # Replace 'Default' with the correct profile

# Set up the ChromeDriver using webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the URL in a browser window
driver.get(url)

# Wait for the page to load completely
time.sleep(23)  # Adjust this delay if needed
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# Click "View More" buttons to load all content
try:
    while True:
        # Find the "View More" button
        view_more_buttons = driver.find_elements(By.XPATH, "//span[contains(@class, 'underline') and contains(text(), 'View More')]")

        if not view_more_buttons:
            break  # No more "View More" buttons are available, break the loop

        # Loop through the buttons to click and load more content
        for button in view_more_buttons:
            try:
                # Scroll to the button to ensure it's in view
                driver.execute_script("arguments[0].scrollIntoView(true);", button)

                # Use JavaScript to click the button to avoid click issues
                driver.execute_script("arguments[0].click();", button)

                # Add a delay to allow new content to load
                time.sleep(4)
            except Exception as inner_e:
                print(f"Error occurred while clicking a specific 'View More' button: {inner_e}")

except Exception as e:
    print("Error occurred while clicking 'View More' buttons: ", e)

# Click "View Comments" buttons to expand comments
try:
    previous_comment_count = 0

    while True:  # Keep clicking until no more buttons are found
        # Find all buttons with the chevron down icon for expanding comments
        show_comments_buttons = driver.find_elements(By.XPATH, "//i[contains(@class, 'fa-chevron-down')]")
        print(f"Number of pushable buttons {len(show_comments_buttons)}")

        # Skip the first 4 instances since they aren't comments
        if len(show_comments_buttons) <= 4:
            break  # If there aren't more than 4 buttons, exit the loop
        show_comments_buttons = show_comments_buttons[4:]

        ## Skip the 100 instances to see if I can get the rest of the file
        #if len(show_comments_buttons) <= 170:
        #    break  # If there aren't more than 130 buttons, exit the loop
        #show_comments_buttons = show_comments_buttons[170:]

        
        # List to store the last 500 characters of the page source before clicking a button
        click_contexts = []
        
        # Click on each button found, starting from the fifth button
        for index, button in enumerate(show_comments_buttons):
            try:
                # Scroll to the button to ensure it's in view
                driver.execute_script("arguments[0].scrollIntoView(true);", button)

                # Capture the button HTML and its context before clicking
                button_html = button.get_attribute('outerHTML')
                current_page_source = driver.page_source
                context_snippet = current_page_source[-500:]
                click_contexts.append({
                    "button_index": index,
                    "button_html": button_html,
                    "context_snippet": context_snippet,
                })

                # Log information for debugging
                print(f"\nAttempting to click button #{index + 1} with HTML:\n{button_html}\n")

                # Use JavaScript to click the button to avoid click issues
                driver.execute_script("arguments[0].click();", button)

                # Wait for new comments to load by checking for any new 'viewComment' class elements
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "viewComment"))
                )

                # Add a delay to allow comments to fully load
                time.sleep(2)

            except Exception as inner_e:
                print(f"Error occurred while clicking a specific comment button #{index + 1}: {inner_e}")

        # Debugging information: Print the last context snippet
        if click_contexts:
            print("\nThe last context before a button click was:")
            print(f"Button Index: {click_contexts[-1]['button_index']}")
            print(f"Button HTML: {click_contexts[-1]['button_html']}")
            print(f"Context Snippet: {click_contexts[-1]['context_snippet']}")
        else:
            print("\nNo buttons were successfully clicked.")

        # Extract the page source after clicking to see if the number of comments has increased
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        comments = soup.find_all('div', class_='viewComment')

        # Break the loop if no new comments were found
        if len(comments) == previous_comment_count:
            break
        previous_comment_count = len(comments)

except Exception as e:
    print("Error occurred while expanding comments: ", e)

# Extract the page source and parse it with BeautifulSoup
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Close the browser window
driver.quit()




In [80]:
# Save the page source to a file
with open("page_source.html", "w", encoding="utf-8") as file:
    file.write(page_source)



In [ ]:
# Later, to reload the page source and parse it:
with open("page_source.html", "r", encoding="utf-8") as file:
    saved_page_source = file.read()
    soup = BeautifulSoup(saved_page_source, 'html.parser')

# Now, `soup` is ready for parsing.


BELOW is not in use, see caringbridge_parser.ipynb next

# Extract comments using BeautifulSoup
comments = soup.find_all('div', class_='viewComment')
if comments:
    for index, comment in enumerate(comments, start=1):
        # Extract commenter's name
        commenter_tag = comment.find('span', class_='commentHeader')
        if commenter_tag:
            commenter_name = commenter_tag.find('strong').get_text(strip=True)
            print(f"Comment {index} by {commenter_name}:")
        
        # Extract comment content
        comment_body_tag = comment.find('div', class_='commentBody')
        if comment_body_tag:
            comment_content = comment_body_tag.get_text(strip=True)
            print(f"Content: {comment_content}")
        
        print("=" * 50)

    print(f"Total comments extracted: {len(comments)}")
else:
    print("No comments found.")

In [92]:
for line in page_source.splitlines():
    if '<div class="jsx-90dd3d5d3d6464a0 quadLayout">' in line:
        print(line[:50])

}</style><style type="text/css" data-styled-jsx=""
<p>Thank you all for your continued support. Lets 
</div></div><span class="chakra-link readMoreToggl
<p>When Ada was alive we had a dinner time traditi
<p>Love Tami and Joel</p></div><span class="chakra
<p>As always, thanks for your continued support.</
<p>We couldn't focus on her and our other kids as 
<p>-Joel</p></div><span class="chakra-link readMor
<p>If you read this whole post you really need to 
<p>Oliver, Ellie and Joel headed back to WI after 
<p>We keep meaning to write a long and heartfelt p
<p>We also want to post about the blood and bone m
<p>- Joel</p></div><span class="chakra-link readMo
<p>Other fun things happening soon: Ada's wish thr
</div></div><span class="chakra-link readMoreToggl
<p>Everyone's love and support means the world to 
<p>-Holly Ertmer (Tami's sister)</p></div><span cl
<p>I love Ada so much and am so thankful that she 
<p>Memphish, TN 38105</p></div><span class="chakra
<p>-Tami</p></div><span class="

from bs4 import BeautifulSoup

# Load the saved HTML file into BeautifulSoup
with open(file_path, 'r', encoding='utf-8') as file:
    page_content = file.read()

soup = BeautifulSoup(page_content, 'html.parser')

# Find all comment elements based on the relevant class
comments = soup.find_all('div', class_='viewComment')

# Extract metadata from each comment
for comment in comments:
    # Extract commenter name and date
    commenter_tag = comment.find('span', class_='commentHeader')
    if commenter_tag:
        # Extract the commenter's name
        commenter_name_tag = commenter_tag.find('strong')
        commenter_name = commenter_name_tag.get_text(strip=True) if commenter_name_tag else "Unknown"

        # Extract the date by splitting the remaining text
        comment_date_text = commenter_tag.get_text(strip=True)
        comment_date = comment_date_text.split('—')[-1].strip() if '—' in comment_date_text else "Unknown"
    else:
        commenter_name = "Unknown"
        comment_date = "Unknown"
    
    # Extract comment content
    comment_body_tag = comment.find('div', class_='commentBody')
    comment_content = comment_body_tag.get_text(strip=True) if comment_body_tag else "No content"

    # Print the extracted comment information
    print(f"Commenter: {commenter_name}")
    print(f"Date: {comment_date}")
    print(f"Comment: {comment_content}")
    print("=" * 50)


## Extract Reactions

From the below need to extract "Praying", "Hearts", "41" ....I could try to extract names of who did each eventually
Can also extract number of comments from the below line = "12"

<div data-qa="commentCountButtons" class="jsx-3352288381 countButtons"><button data-qa="reactionCounter-d3aa63a0-b7c6-3049-8d14-f9915ceaa339" class="jsx-1378e3855087140a reactionCounter button--isClear"><div data-qa="rc-icon-Heart" class="jsx-1378e3855087140a reactionCounter__emoji"><img src="/img/emojis/emoji-heart.svg" alt="Heart icon" class="jsx-1378e3855087140a"></div><div data-qa="rc-icon-Prayer" class="jsx-1378e3855087140a reactionCounter__emoji"><img src="/img/emojis/emoji-pray.svg" alt="Folded hands icon" class="jsx-1378e3855087140a"></div><span data-qa="rc-count" class="jsx-1378e3855087140a reactionCounter__count"><span class="jsx-1378e3855087140a sr-only">Reactions count: </span>41</span></button><button class="tertiaryButton button--isClear hideOutlineForMouse noPrint jsx-1878294938 commentCounter withoutIcon" tabindex="0" data-qa="toggleComments-d3aa63a0-b7c6-3049-8d14-f9915ceaa339" type="button"><span class="jsx-1878294938 flexOrder"><span class="jsx-1878294938 underline"><span class="jsx-ea16c7692e41b271 ">12 comments</span> <i class="fa-chevron-down fa-light" aria-hidden="true"></i></span>&nbsp;&nbsp;</span></button></div>

Another example below should yield "Hearts", "46"
Number of comments = "15"

<div data-qa="commentCountButtons" class="jsx-3352288381 countButtons"><button data-qa="reactionCounter-2d7616a6-772f-3407-8104-b1cd98b9018f" class="jsx-1378e3855087140a reactionCounter button--isClear"><div data-qa="rc-icon-Heart" class="jsx-1378e3855087140a reactionCounter__emoji"><img src="/img/emojis/emoji-heart.svg" alt="Heart icon" class="jsx-1378e3855087140a"></div><span data-qa="rc-count" class="jsx-1378e3855087140a reactionCounter__count"><span class="jsx-1378e3855087140a sr-only">Reactions count: </span>46</span></button><button class="tertiaryButton button--isClear hideOutlineForMouse noPrint jsx-1878294938 commentCounter withoutIcon" tabindex="0" data-qa="toggleComments-2d7616a6-772f-3407-8104-b1cd98b9018f" type="button"><span class="jsx-1878294938 flexOrder"><span class="jsx-1878294938 underline"><span class="jsx-ea16c7692e41b271 ">15 comments</span> <i class="fa-chevron-up fa-light" aria-hidden="true"></i></span>&nbsp;&nbsp;</span></button></div>